# Center Inter-comparison Demonstration

This tutorial demonstration analyzes the differences between RO retrievals 
of COSMIC-1 data as produced by the Radio Occultation Meteorology Satellite 
Application Facility (ROM SAF) and by the COSMIC Project Office at the 
University Corporation for Atmospheric Research (UCAR). 

_This notebook works with version 1.0.6 or later of awsgnssroutils. If you 
experience faults, it is most likely that you have to update that package 
by pip._

### Set defaults and populate metadata. 

If you haven't previously done so, be sure to set defaults for the metadata (and data) and pre-populate the metadata database. This could take a while, but it's well worth your time for the sake of very efficient metadata queries in future work. 

In [ ]:
if False: 
    from awsgnssroutils.database import setdefaults, populate
    import os

    HOME = os.path.expanduser("~")
    rometadata = os.path.join( HOME, "local", "rometadata" )
    rodata = os.path.join( HOME, "Data", "rodata" )

    #  Define defaults for awsgnssroutils 

    setdefaults( repository=rometadata, rodata=rodata, version="v1.1" )
    populate()


### Imports

Import modules necessary for execution and set defaults. Be certain that the 
resources file has been created by awsgnssroutils.database.setdefaults. 

In [ ]:
import os
import json
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from netCDF4 import Dataset
import cartopy
from awsgnssroutils.database import RODatabaseClient

day = datetime( year=2010, month=6, day=24 )
json_savefile = "center_intercomparison_stats.{:4d}-{:02d}-{:02d}.json".format( day.year, day.month, day.day )

#  Physical constants.

gravity = 9.80665           # WMO standard gravity [J/kg/m]
k1 = 77.6e-2                # First term in refractivity equation [N-units K/Pa]
muvap = 18.015e-3           # Mean molecular mass of water vapor [kg/mole]
mudry = 28.965e-3           # Mean molecular mass of dry air [kg/mole]

#  Instantiate the database. 

db = RODatabaseClient()

## Data analysis

Analyze COSMIC-1 differences in retrieval for one day, given by *day*. First, define 
a function that returns a masked array when the independent coordinate is out of 
bounds. 

In [ ]:
def masked_interpolate( x_in, y_in, x_out ):
    """Performed a masked cubic spline interpolation. The output array is masked
    where an interpolation is impossible."""

    #  Strip out masked values in the input arrays.

    if x_in.mask.size == 1:
        xmask = np.repeat( x_in.mask, x_in.size )
    else:
        xmask = x_in.mask

    if y_in.mask.size == 1:
        ymask = np.repeat( y_in.mask, y_in.size )
    else:
        ymask = y_in.mask

    i = np.argwhere( np.logical_not( np.logical_or( xmask, ymask ) ) ).squeeze()
    xi, yi = x_in[i], y_in[i]

    #  Form the interpolator.

    fill_value = -1.332e21
    finterp = interp1d( xi, yi, kind='cubic', bounds_error=False, fill_value=fill_value )

    #  Do the interpolation.

    y = finterp( x_out )

    #  Mask the output array.

    y_out = np.ma.masked_where( y == fill_value, y )

    #  Done.

    return y_out


Now, analyze data. 

In [ ]:
# Establish the datetimerange. 

datetime1 = day
datetime2 = day + timedelta(days=1)

# Query the database for refractivityRetrieval matchups, atmosphericRetrieval matchups. 

occs1 = db.query( missions="cosmic1", 
                datetimerange=(datetime1.isoformat(),datetime2.isoformat()), 
                availablefiletypes=("ucar_refractivityRetrieval","romsaf_refractivityRetrieval") )
occs2 = db.query( missions="cosmic1", 
                datetimerange=(datetime1.isoformat(),datetime2.isoformat()), 
                availablefiletypes=("ucar_atmosphericRetrieval","romsaf_atmosphericRetrieval") )

#  Independent coordinate for bending angle, in meters.

common_impactHeight = np.arange( 0.0, 60.0001e3, 100 )

#  Independent coordinate for log-refractivity, dry temperature, temperature,
#  and specific humidity, in meters.

common_geopotentialHeight = np.arange( 0.0, 60.0001e3, 200 )

#  Analysis of refractivityRetrieval files. Download the refractivityRetrieval
#  files common to UCAR and ROM SAF and compare bending angle, log-refractivity,
#  dry temperature.

#  Initialize difference statistics.

diffs_bendingAngle = [ [] for i in range(len(common_impactHeight)) ]
diffs_logrefractivity = [ [] for i in range(len(common_geopotentialHeight)) ]
diffs_dryTemperature = [ [] for i in range(len(common_geopotentialHeight)) ]

#  Download. 

print( "Analyzing refractivityRetrieval files" )
print( "First download data files" )

files = { "ucar": None, "romsaf": None }
for center in files.keys(): 
    files[center] = occs1.download( center+"_refractivityRetrieval" )
    nfiles = len( files[center] )
print( f"Number of files for refractivity inter-comparison = {nfiles}" )

#  Loop over soundings. 

print( "Looping over refractivityRetrieval data files" )

for isounding in tqdm(range(nfiles),desc="Sounding..."):

    diff_bendingAngle = np.ma.array( np.zeros( len(common_impactHeight) ), mask=False )
    diff_logrefractivity = np.ma.array( np.zeros( len(common_geopotentialHeight) ), mask=False )
    diff_dryTemperature = np.ma.array( np.zeros( len(common_geopotentialHeight) ), mask=False )
    status = "success"

    for center in [ "ucar", "romsaf" ]:

        #  Open NetCDF file.

        data = Dataset( files[center][isounding], 'r' )

        #  L1 bending angle and impact parameter.

        input_bendingAngle = data.variables['bendingAngle'][:]
        input_impactParameter = data.variables['impactParameter'][:]

        #  Get radius of curvature in order to compute impact height from
        #  impact parameter.

        radiusOfCurvature = data.variables['radiusOfCurvature'].getValue()

        #  Compute impact height from impact parameter by subtracting the local
        #  radius of curvature for this RO sounding.

        input_impactHeight = input_impactParameter - radiusOfCurvature

        #  Interpolate bending angle onto the common impact heights.

        bendingAngle = masked_interpolate( input_impactHeight, input_bendingAngle,
                common_impactHeight )

        #  Interpolate log-refractivity onto the common geopotential heights.

        input_geopotentialHeight = data.variables['geopotential'][:] / gravity
        input_logrefractivity = np.log( data.variables['refractivity'][:] )

        #  Interpolate log-refractivity onto the common geopotential heights.

        logrefractivity = masked_interpolate( input_geopotentialHeight, input_logrefractivity,
                common_geopotentialHeight )

        #  Generate dry temperature.

        input_dryPressure = data.variables['dryPressure'][:]
        input_dryTemperature = input_dryPressure / np.exp( input_logrefractivity ) * k1

        #  Interpolate dry temperature onto the common geopotential heights.

        dryTemperature = masked_interpolate( input_geopotentialHeight, input_dryTemperature,
                common_geopotentialHeight )

        #  Close and remove input file.

        data.close()
 
        #  Calculate difference between UCAR and ROM SAF.

        if center == "ucar":
            diff_bendingAngle += bendingAngle
            diff_logrefractivity += logrefractivity
            diff_dryTemperature += dryTemperature
        elif center == "romsaf":
            diff_bendingAngle -= bendingAngle
            diff_logrefractivity -= logrefractivity
            diff_dryTemperature -= dryTemperature

    #  Record the differences between UCAR and ROM SAF for bending angle,
    #  log-refractivity, and dry temperature.

    for i in np.argwhere( np.logical_not( diff_bendingAngle.mask ) ).squeeze():
        diffs_bendingAngle[i].append( diff_bendingAngle[i] )

    for i in np.argwhere( np.logical_not( diff_logrefractivity.mask ) ).squeeze():
        diffs_logrefractivity[i].append( diff_logrefractivity[i] )

    for i in np.argwhere( np.logical_not( diff_dryTemperature.mask ) ).squeeze():
        diffs_dryTemperature[i].append( diff_dryTemperature[i] )

#  Analysis of atmosphericRetrieval files. Download the atmosphericRetrieval
#  files common to UCAR and ROM SAF and compare bending angle, log-refractivity,
#  dry temperature.

print( "Analyzing atmosphericRetrieval files" )
print( "Download data files" )

files = { "ucar": None, "romsaf": None }
for center in files.keys(): 
    files[center] = occs2.download( center+"_atmosphericRetrieval" )
    nfiles = len( files[center] )
print( f"Number of files for atmospheric inter-comparison = {nfiles}" )

#  Initialize difference statistics.

diffs_temperature = [ [] for i in range(len(common_geopotentialHeight)) ]
diffs_specificHumidity = [ [] for i in range(len(common_geopotentialHeight)) ]

#  Loop over atmosphericRetrieval files.

for isounding in tqdm(range(nfiles),desc="Sounding..."):

    diff_temperature = np.ma.array( np.zeros( len(common_geopotentialHeight) ), mask=False )
    diff_specificHumidity = np.ma.array( np.zeros( len(common_geopotentialHeight) ), mask=False )

    for center in [ "ucar", "romsaf" ]:

        #  Open NetCDF file.

        data = Dataset( files[center][isounding], 'r' )

        #  Interpolate temperature onto the common geopotential heights.

        input_geopotentialHeight = data.variables['geopotential'][:] / gravity
        input_temperature = data.variables['temperature'][:]
        temperature = masked_interpolate( input_geopotentialHeight, input_temperature,
                    common_geopotentialHeight )

        #  Compute specific humidity.

        input_pressure = data.variables['pressure'][:]
        input_waterVaporPressure = data.variables['waterVaporPressure'][:]
        input_specificHumidity = ( muvap * input_waterVaporPressure ) / \
                    ( mudry * ( input_pressure - input_waterVaporPressure ) \
                    + muvap * input_waterVaporPressure )

        #  Interpolate specific humidity onto the common geopotential heights.

        input_geopotentialHeight = data.variables['geopotential'][:] / gravity
        specificHumidity = masked_interpolate( input_geopotentialHeight,
                    input_specificHumidity, common_geopotentialHeight )

        #  Close and remove input file.

        data.close()

        #  Calculate difference between UCAR and ROM SAF.

        if center == "ucar":
            diff_temperature += temperature
            diff_specificHumidity += specificHumidity
        elif center == "romsaf":
            diff_temperature -= temperature
            diff_specificHumidity -= specificHumidity

    #  Record the differences between UCAR and ROM SAF for temperature and
    #  specificHumidity.

    for i in np.argwhere( np.logical_not( diff_temperature.mask ) ).squeeze():
        diffs_temperature[i].append( diff_temperature[i] )

    for i in np.argwhere( np.logical_not( diff_specificHumidity.mask ) ).squeeze():
        diffs_specificHumidity[i].append( diff_specificHumidity[i] )

#  Save to output file.

output_dict = {
        'common_impactHeight': list( common_impactHeight ),
        'common_geopotentialHeight': list( common_geopotentialHeight ),
        'diffs_bendingAngle': diffs_bendingAngle,
        'diffs_logrefractivity': diffs_logrefractivity,
        'diffs_dryTemperature': diffs_dryTemperature,
        'diffs_temperature': diffs_temperature,
        'diffs_specificHumidity': diffs_specificHumidity }

print( f"Storing data in {json_savefile}." )

with open( json_savefile, 'w' ) as out:
    json.dump( output_dict, out, indent="  " )


## Results figure

### matplotlib settings

Define defaults for matplotlib plotting. 


In [ ]:
axeslinewidth = 0.5
plt.rcParams.update( {
  'font.family': "Times New Roman",
  'font.size': 12,
  'font.weight': "normal",
  'text.usetex': False,
  'xtick.major.width': axeslinewidth,
  'xtick.minor.width': axeslinewidth,
  'ytick.major.width': axeslinewidth,
  'ytick.minor.width': axeslinewidth,
  'axes.linewidth': axeslinewidth } )

### Plot the statistics of inter-center comparison analysis

In [ ]:
#  Read JSON file.

with open( json_savefile, 'r' ) as e:
    data = json.load( e )

#  Set up figure.

plt.clf()
fig = plt.figure( figsize=(9,6.5) )

#  First axis: log-refractivity comparison.

green_diamond = { 'markerfacecolor': "g", 'markeredgecolor': "g", 
        'marker': "D", 'markersize': 1.0 }
kwargs = { 'vert': False, 'whis': (5,95), 'flierprops': green_diamond,
        'manage_ticks': False, 'widths': 0.4 }

ax = fig.add_axes( [ 0.12, 0.14, 0.86, 0.80 ] )

ytickv = np.arange( 0.0, 20.01, 5 )
ax.set_yticks( ytickv )
ax.set_ylim( ytickv.min(), ytickv.max() )
ax.yaxis.set_minor_locator( MultipleLocator(1) )
ax.set_ylabel( "Impact Height [km]" )

xtickv = np.arange( -4, 4.01, 2 )
ax.set_xticks( xtickv )
ax.set_xlim( xtickv.min(), xtickv.max() )
ax.xaxis.set_minor_locator( MultipleLocator(0.5) )
ax.set_xlabel( "Bending Angle Diffs [millirads]" )

#  Subset the data by level to every 1-km.

resolution = 1.0e3
da = np.abs( data['common_impactHeight'][1] - data['common_impactHeight'][0] )
nskip = int( resolution / da )

subset_diffs_bendingangle = []
subset_impactHeights = []

for ia in range( 0, len( data['common_impactHeight'] ), nskip ):
    subset_impactHeights.append( data['common_impactHeight'][ia] )
    subset_diffs_bendingangle.append(
            np.array( data['diffs_bendingAngle'][ia] ) * 1.0e3 )
subset_impactHeights = np.array( subset_impactHeights ) / 1000

#  Box and whisker plot.

ax.boxplot( subset_diffs_bendingangle,
        positions=subset_impactHeights, **kwargs )

#  Save to encapsulated postscript file.

epsfile = "center_intercomparison.{:4d}-{:02d}-{:02d}.eps".format( 
    day.year, day.month, day.day )

if False: 
    print( f"Saving figure to {epsfile}." )
    fig.savefig( epsfile, format="eps" )
else: 
    plt.show()
